In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
# !wget https://f000.backblazeb2.com/file/malaya-model/v39/paraphrase/base.pb
# !wget https://f000.backblazeb2.com/file/malaya-model/v39/paraphrase/small.pb

In [4]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.compat.v1.set_random_seed(0)

In [5]:
pbs = glob('*.pb')
pbs

['small.pb', 'base.pb']

In [6]:
import tensorflow_text
import tf_sentencepiece

In [7]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
        
    print(pb)
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['Placeholder', 'Placeholder_2'],
                                           ['greedy', 'beam', 'nucleus'], transforms)
    
    with tf.compat.v1.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.compat.v1.gfile.GFile.
small.pb
base.pb


In [8]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.compat.v1.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.compat.v1.Graph().as_default() as graph:
        tf.compat.v1.import_graph_def(graph_def)
    return graph

In [9]:
quantized = glob('*.pb.quantized')
quantized

['small.pb.quantized', 'base.pb.quantized']

In [14]:
!rm *.pb*

In [ ]:
# converter = @@#lite.TFLiteConverter.from_frozen_graph(
#     graph_def_file='test.pb',
#     input_arrays=['Placeholder', 'Placeholder_1'],
#     input_shapes={'Placeholder' : [None, 512], 'Placeholder_1': [None, 512]},
#     output_arrays=['logits'],
# )
# # converter.allow_custom_ops=True

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.compat.v1.float16]
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# tflite_model = converter.convert()

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.compat.v1.float16]
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-float16.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.compat.v1.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-hybrid.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# interpreter = tf.compat.v1.lite.Interpreter(model_path='tiny-bert-sentiment-hybrid.tflite')
# interpreter.allocate_tensors()